In [1]:
import pandas as pd
import os
path='dataset_2/tcpflow'
files=os.listdir(path)
dataframe=pd.DataFrame()

#汇总tcpflow数据，总量30446655
for file in files:  
    f=pd.read_csv(path+'/'+file)
    print(file)
    
    dataframe=dataframe.append(f)
print(dataframe.head())

part-00000.csv
part-00001.csv
part-00002.csv
part-00003.csv
part-00004.csv
part-00005.csv
part-00006.csv
part-00007.csv
part-00008.csv
part-00009.csv
   Unnamed: 0  destination_ip  destination_port  downlink_length protocol  \
0           0   118.194.56.52               443             4729      ssl   
1           1   202.75.220.12              5226              373  unknown   
2           2  183.129.205.92              5227              397  unknown   
3           3    110.76.20.18               443              547  unknown   
4           4     10.24.64.39              6682              125    redis   

               record_time      source_ip  uplink_length  
0  2019-05-30 19:58:58.035  10.56.131.163           2164  
1  2019-05-30 22:27:23.798  10.54.251.114            412  
2  2019-05-30 22:27:23.905  10.54.251.114            425  
3  2019-05-30 22:28:41.793  10.54.251.114            445  
4  2019-05-30 23:42:45.509  10.49.137.136           3501  


In [24]:
#----------------------------------------掩码16位的IP段内主机数对比----------------------------------------#

import os
import pandas as pd

def find_last(s, t):
    
    last_pos = -1
    
    while True:
        
        pos = s.find(t, last_pos + 1)
        
        if pos == -1:
            
            return last_pos
        
        else:
            
            last_pos = pos

kid={}#每个内网网段以及包含的主机
outnet={}#外网每个IP以及出现次数

#--------------------------对每个IP 掩16位,并统计每个子网出现次数-------------------------#
#     传入的参数为dataframe['dest_ip'],dataframe['source_ip']
def findchildnet_16(D,S):
    
    dest=list(D)
    
    sou=list(S)
    
    dest2=[]#返回掩后的destination——IP
    
    sou2=[]#返回掩后的source——IP
    
    for d in dest:
        
        if d[:3]=='10.':
            
            d_temp=d
        
            last=find_last(d,'.')
            
            d=d[:last]
            
            last2=find_last(d,'.')
            
            d=d[:last2]
            
            d=d+'.0.0'
            
            dest2.append(d)
            
            if not d in kid:
                
                kid[d]=[]
                
                kid[d].append(d_temp)
                
            else:
                
                #kid[d]=kid[d]+1
                if d_temp not in kid[d]:
                
                    kid[d].append(d_temp)
            
            
        else:
            
            dest2.append(d)
            
            if not d in outnet:
                
                outnet[d]=1
                
            else:
                
                outnet[d]=outnet[d]+1
            
    for s in sou:
        
        if s[:3]=='10.':
            
            s_temp=s
        
            last=find_last(s,'.')
            
            s=s[:last]
            
            last2=find_last(s,'.')
            
            s=s[:last2]
            
            s=s+'.0.0'
            
            sou2.append(s)
            
            if not s in kid:
                
                kid[s]=[]
                
                kid[s].append(s_temp)
                
            else:
                
                if s_temp not in kid[s]:
                
                    kid[s].append(s_temp)
            
            
        else:
            
            sou2.append(s)
            
            if not s in outnet:
                
                outnet[s]=1
                
            else:
                
                outnet[s]=outnet[s]+1
            
    return(dest2,sou2)

In [34]:
# D,S=findchildnet_16(temp['destination_ip'],temp['source_ip'])
# temp['dest_group']=D
# temp['source_group']=S
# print(len(kid),len(outnet))
# tmp=pd.DataFrame()

# ip_covered=list(kid.keys())

# tmp=temp[(temp['dest_group'].isin(ip_covered))&(temp['source_group'].isin(ip_covered))]
# light_tmp=pd.DataFrame()
light_tmp=unRedundant(tmp)#去重后的表，为了GEPHI
ID0,Source1,Target1,ID1,Weight,Label1=IP2ID(light_tmp['destination_ip'],light_tmp['source_ip'],light_tmp['flow'])
print(len(ID0))
ftp_control=buildnode(ID0)
ftp_control_edge=buildedge(Source1,Target1,ID1,Weight,Label1)
ftp_control_edge.head()

MemoryError: 

In [28]:
import json

kidnet=pd.DataFrame()

def dict2json(Dict,json_file):
    
    with open(json_file,'w') as f:
        
        f.write(json.dumps(Dict))
            
dict2json(outnet,'outnet.txt')    

In [14]:
#--------------------------------------生成GEPHI需要的数据格式--------------------------------------------#
#传入的参数：原始数据的destination_ip,source_ip,weight可以自己定义是流量总和或上下行..label也自己定义。
#这里除了label传进来的都应该是dataframe的列。
def IP2ID(des,sou,wei):
    ID={}#IP与对应的ID
    
    Target=[]
    Source=[]
    
    ID2=[]#edge表的另一行ID
    weight=[]
    
    count=0
    id1=0
    
    de_ip=list(des)
    sou_ip=list(sou)
    weig=list(wei)
    
    
    for ip in de_ip:
        ID2.append(count)
        count=count+1
        #记录Ip对应的ID
        if not ip in ID:
            ID[ip]=id1
            id1=id1+1
        #为了edge表做准备
        Target.append(ID[ip])            
    
    for ip in sou_ip: 
        #记录Ip对应的ID
        if not ip in ID:
            ID[ip]=id1
            id1=id1+1
        #为了edge表做准备
        Source.append(ID[ip])
        
    for w in weig:
        weight.append(w)
    Label=None

    return ID,Source,Target,ID2,weight,Label


#-------------------建立点表----------------------#
#传的参数是IP2ID返回的ID
def buildnode(i):    
    t=pd.DataFrame({'ID':list(i.values()),'Label':list(i.keys())})
    t['Modularity']=None
    t['Class']=None
    return t

#------------------建立边表----------------------#
#传的参数是IP2ID返回的Source,Target，ID2,weight，Label
def buildedge(S,T,i,W,L):
    edge=pd.DataFrame({'Source':S,'Target':T})
    edge['Type']='Directed'
    edge['ID']=i
    edge['Label']=L
    edge['Weight']=W
    return edge

In [33]:
dictionary={}
#-----------------------去重统计每一对的总的上下行流量，但是会丢失端口号等信息--------------------#
def unRedundant(DF):
    temp0=DF
    dip=[]
    sip=[]
    ip=pd.DataFrame()
    ip1=pd.DataFrame()
    
    
    
    ip['destination_ip']=temp0['destination_ip']
    ip['source_ip']=temp0['source_ip']
    ip['uplink_length']=temp0['uplink_length']
    ip['downlink_length']=temp0['downlink_length']
    ip['flow']=temp0['flow']
    
    up=ip.groupby(['destination_ip','source_ip'])[['uplink_length']].sum()
    down=ip.groupby(['destination_ip','source_ip'])[['downlink_length']].sum()
    flow=ip.groupby(['destination_ip','source_ip'])[['flow']].sum()
    degree=[]
    
    up0=list(up['uplink_length'])
    down0=list(down['downlink_length'])
    flow0=list(flow['flow'])
    
    i=ip.groupby(['destination_ip','source_ip'])
    for name1,group in i:
        name=list(name1)
        dip.append(name[0])
        sip.append(name[1])
        degree.append(group.shape[0])

    ip1['destination_ip']=dip
    ip1['source_ip']=sip
    ip1['downlink_length']=down0
    ip1['uplink_length']=up0
    ip1['flow']=flow0

    return ip1


In [19]:
temp=pd.DataFrame()
temp=dataframe
#------------------------------------把UPLINK_LENGTH,DOWNLINK_LENGTH加起来---------------------------------#
def count_flow(temp):
    up=list(temp['downlink_length'])
    down=list(temp['uplink_length'])
    flow=[]
    for i in range(len(up)):
        flow.append(up[i]+down[i])
    print(len(flow))
    return flow
    #temp['flow']=flow

# ftp_control.to_csv('csv/ftp_control_node.csv',index=False)
# ftp_control_edge.to_csv('csv/ftp_control_edge.csv',index=False)

50422


,Source,Target,Type,ID,Label,Weight
0,45063,0,Directed,0,None,6893
1,114,1,Directed,1,None,785
2,114,2,Directed,2,None,822
3,114,3,Directed,3,None,992
4,16532,4,Directed,4,None,3626


In [3]:
#-------------------------按IP切分原数据--------------------------#
#输入的参数：IP必须是一个list,DF为原始的总数据
def cut_by_IP(ip,DF):
    data=pd.DataFrame()
    data1=pd.DataFrame()
    data2=pd.DataFrame()
    
    data=DF[DF['destination_ip'].isin(ip)]
    data1=DF[DF['source_ip'].isin(ip)]
    
    data2=data2.append(data)
    data2=data2.append(data1)
    return data2


#-------------------------按协议分的不掩码IP源数据--------------------------#
#输入的参数：IP和protocol都必须是一个list,DF为原始的总数据
def cut_by_IP_pro(ip,protocol,DF):
    data=pd.DataFrame()
    data1=pd.DataFrame()
    data2=pd.DataFrame()
    data_true=pd.DataFrame()
    
    data=DF[DF['destination_ip'].isin(ip)]
    data1=DF[DF['source_ip'].isin(ip)]
    
    data2=data2.append(data)
    data2=data2.append(data1)
    
    data_true=data2[data2['protocol'].isin(protocol)]
    return data2

In [7]:
# f=open('tmp.txt')
# ips=[]
# ip_pro={}
# for lines in f:
#     l=lines.strip('\n')
#     ips.append(l)

# for ip in ips:
#     ip_list=[ip]
#     tempt=pd.DataFrame()
#     tempt=cut_by_IP(ip_list,dataframe)
#     pros=set(list(tempt['protocol']))
#     ip_pro[ip]=pros
#     print(ip)
# print(ip_pro)

result=pd.DataFrame()
result['ip']=list(ip_pro.keys())
result['pros']=list(ip_pro.values())
result.to_csv('ip_pros.csv',index=False)